In [ ]:
import os
import json

from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
        azure_deployment=os.environ["AZURE_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )

In [ ]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

In [ ]:
from promptflow.client import load_flow

# load apology evaluatorfrom prompty
groundedness_eval = load_flow(source="prompty/groundedness.prompty", model={"configuration": model_config})
fluency_eval = load_flow(source="prompty/fluency.prompty", model={"configuration": model_config})
coherence_eval = load_flow(source="prompty/coherence.prompty", model={"configuration": model_config})
relevance_eval = load_flow(source="prompty/relevance.prompty", model={"configuration": model_config})

In [ ]:

results = []

for index, row in df.iterrows():
    customer_id = row['customerId']
    question = row['question']
    
    # Run contoso-chat/chat_request flow to get response
    response = get_response(customer_id=customer_id, question=question)
    
    # Add results to list
    result = {
        'customerId': customer_id,
        'question': question,
        'response': response
    }
    results.append(result)

# Save results to a JSONL file
with open('results.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

In [ ]:
from promptflow.evals.evaluate import evaluate
#from promptflow.evals.evaluators.content_safty import violence_eval

result = evaluate(
    data="results.jsonl",
    evaluators={
        #"violence": violence_eval,
        "groundedness": groundedness_eval,
        "fluecy": fluency_eval,
        "coherence": coherence_eval,
        "relevence": relevance_eval,
    },
    # column mapping
    evaluator_config={
        "default": {
            "question": "${data.question}",
            "answer": "${run.outputs.answer}",
            "context": "${run.outputs.context}",
        }
    }
)

In [ ]:
from IPython.display import display, JSON

display(JSON(result))

In [ ]:
# Initialize Project Scope
#project_scope = {config.json}

In [ ]:
# Check the results using Azure AI Studio UI
#print(result["studio_url"])